# About this notebook
- PyTorch Resnet18 starter code
- [train kernel](https://www.kaggle.com/yasufuminakama/herbarium-2020-pytorch-resnet18-train) -> inference kernel

If this notebook is helpful, feel free to upvote :)

# Library

In [ ]:
import os
import numpy as np 
import pandas as pd 
import json

In [ ]:
os.listdir('../input/herbarium-2020-fgvc7')

# Data Loading

In [ ]:
%%time

with open('../input/herbarium-2020-fgvc7/nybg2020/train/metadata.json', "r", encoding="ISO-8859-1") as file:
    train = json.load(file)

train_img = pd.DataFrame(train['images'])
train_ann = pd.DataFrame(train['annotations']).drop(columns='image_id')
train_df = train_img.merge(train_ann, on='id')
train_df.head()

In [ ]:
%%time

with open('../input/herbarium-2020-fgvc7/nybg2020/test/metadata.json', "r", encoding="ISO-8859-1") as file:
    test = json.load(file)

test_df = pd.DataFrame(test['images'])
test_df.head()

In [ ]:
sample_submission = pd.read_csv('../input/herbarium-2020-fgvc7/sample_submission.csv')
sample_submission.head()

# TARGET ("category_id")

In [ ]:
train_df['category_id'].value_counts()

- imbalance
- 32093 targets

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(train_df['category_id'])
train_df['category_id_le'] = le.transform(train_df['category_id'])
class_map = dict(sorted(train_df[['category_id_le', 'category_id']].values.tolist()))

# Library

In [ ]:
# ====================================================
# Library
# ====================================================

import sys

import gc
import os
import random
import time
from contextlib import contextmanager
from pathlib import Path

import cv2
from PIL import Image
import numpy as np
import pandas as pd
import scipy as sp

import sklearn.metrics

from functools import partial

import torch
import torch.nn as nn
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader, Dataset

from albumentations import Compose, Normalize, Resize
from albumentations.pytorch import ToTensorV2

 #about torch...
import torch
import torch.nn as nn
import torch
import torch.optim as optim
import torch.nn.functional as F
from PIL import Image
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset

#using numpy
import numpy as np

#for data load or save
import pandas as pd

#visualize some datasets
import matplotlib.pyplot as plt

#check our work directory
import os

#to unzip datasets
import zipfile

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

# Utils

In [ ]:
# ====================================================
# Utils
# ====================================================

@contextmanager
def timer(name):
    t0 = time.time()
    LOGGER.info(f'[{name}] start')
    yield
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s.')

    
def init_logger(log_file='train.log'):
    from logging import getLogger, DEBUG, FileHandler,  Formatter,  StreamHandler
    
    log_format = '%(asctime)s %(levelname)s %(message)s'
    
    stream_handler = StreamHandler()
    stream_handler.setLevel(DEBUG)
    stream_handler.setFormatter(Formatter(log_format))
    
    file_handler = FileHandler(log_file)
    file_handler.setFormatter(Formatter(log_format))
    
    logger = getLogger('Herbarium')
    logger.setLevel(DEBUG)
    logger.addHandler(stream_handler)
    logger.addHandler(file_handler)
    
    return logger

LOG_FILE = 'train.log'
LOGGER = init_logger(LOG_FILE)


def seed_torch(seed=777):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

SEED = 777
seed_torch(SEED)

# Dataset

In [ ]:
N_CLASSES = 32093

class TrainDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.df['file_name'].values[idx]
        file_path = f'../input/herbarium-2020-fgvc7/nybg2020/train/{file_name}'
#         image = cv2.imread(file_path)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#         image = Image.fromarray(np.uint8(image))
        image = Image.open(file_path)
        
        if self.transform is not None:
            image = self.transform(image)
            
        label = self.df['category_id'].values[idx]
            
        
        return image, label
    
class TestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.df['file_name'].values[idx]
        file_path = f'../input/herbarium-2020-fgvc7/nybg2020/test/{file_name}'
#         image = cv2.imread(file_path)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#         image = Image.fromarray(np.uint8(image))
        image = Image.open(file_path)
        
        if self.transform is not None:
            image = self.transform(image)
            
            
        
        return image

# Transforms

In [ ]:
HEIGHT = 128
WIDTH = 128

train_transforms =  transforms.Compose([
        transforms.Resize([128,128]),
        transforms.RandomRotation(10, center=(0, 0),expand=True),
        transforms.RandomHorizontalFlip(),
        transforms.Resize([128,128]),
        transforms.ToTensor()
    ])

test_transforms =  transforms.Compose([
        transforms.Resize([128,128]),
        transforms.ToTensor()
])

In [ ]:
from sklearn.model_selection import train_test_split
indexs = [i for i in range(train_df.shape[0])]
train_index, test_index = train_test_split(indexs, test_size=0.2)
train_data = train_df.iloc[train_index]
test_data = train_df.iloc[test_index]

In [ ]:
batch_size = 512


train_dataset = TrainDataset(train_data, transform=train_transforms)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TrainDataset(test_data, transform=train_transforms)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
val_dataset = TestDataset(test_df, transform=test_transforms)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
np.array(val_dataset[1]).shape

# Modeling

In [ ]:
# import torchvision.models as models

# model = models.resnet18(pretrained=True)
# model.avgpool = nn.AdaptiveAvgPool2d(1)
# model.fc = nn.Linear(model.fc.in_features, N_CLASSES)
class MNIST_CNN(nn.Module):
    def __init__(self):
        super(MNIST_CNN,self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,16,kernel_size=3, padding=0,stride=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16,32,kernel_size=3, padding=0,stride=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(32,16,kernel_size=3, padding=0,stride=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
#             nn.MaxPool2d(2)
        )

        self.L1 = nn.Sequential(         
            nn.Linear(in_features=16*3*3,out_features=512),                              
            nn.ReLU()
        )
#         self.L1_1 = nn.Sequential(         
#             nn.Linear(in_features=512,out_features=512),                              
#             nn.ReLU()
#         )
#         self.L1_2 = nn.Sequential(         
#             nn.Linear(in_features=512,out_features=512),                              
#             nn.ReLU()
#         )
        self.L2 = nn.Sequential(         
            nn.Linear(in_features=512,out_features=256),     
            nn.ReLU()                   
        )
        self.L3 = nn.Sequential(         
            nn.Linear(in_features=256,out_features=128),    
            nn.ReLU()                  
        )
        self.L4 = nn.Sequential(         
            nn.Linear(in_features=128,out_features=64),    
            nn.ReLU()                 
        )
        # fully connected layer, output 10 classes
        self.L5 = nn.Sequential(         
            nn.Linear(in_features=64,out_features=N_CLASSES),    
            nn.ReLU()                     
        )
        self.lsfm = nn.Sigmoid()
       
    
    def forward(self, x):
#--------------------------------------------------------------------------
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
#         print(x.shape)             # torch.Size([36, 16, 3, 3])
#--------------------------------------------------------------------------
        x = x.view(x.size(0),-1)
#         print(x.shape)             # torch.Size([36, 144])
#--------------------------------------------------------------------------
        x = self.L1(x)
#         x = self.L1_1(x)
#         x = self.L1_2(x)
        x = self.L2(x)
        x = self.L3(x)
        x = self.L4(x)
        x = self.L5(x) 
        output = self.lsfm(x)     
        return output

# Training Data

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:
model = MNIST_CNN().to(device)
model.train()
optimizer = optim.AdamW(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

In [ ]:
import time
from tqdm import tqdm
epochs=100
history = []
for epoch in range(epochs):
# ---------------------------------------training-------------------------
    epoch_loss = 0
    epoch_accuracy = 0
    model.train()
    for times, (data, label) in enumerate(train_loader):
        data = data.to(device)
        label = label.to(device)
        
        output = model(data)
        loss = criterion(output, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        acc = ((output.argmax(dim=1) == label).float().mean())
        
        epoch_accuracy += acc/len(train_loader)
        epoch_loss += loss/len(train_loader)
        if times% 50 == 0:
            print('Epoch:{} Complete!\nEpoch Loss:{} \nEpoch Accuracy:{}'.format(epoch+1,epoch_loss,epoch_accuracy))
    # ----------------------evaluate--------------------------------
#     model.eval()
#     test_loss = 0
#     test_acc = 0
#     for data, label in (test_loader):
#         data = data.to(device)
#         label = label.to(device)
        
#         output = model(data)
#         loss = criterion(output, label)
#         acc = ((output.argmax(dim=1) == label).float().mean())
        
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
        
#         test_acc += acc/len(test_loader)
#         test_loss += loss/len(test_loader)
        
#     print('Val loss:{}, Val acc:{}'.format(test_loss, test_acc))
#     print('-'*50)
#     print('-'*50)
    
#     history.append([epoch+1,float(epoch_loss.cpu().detach().numpy()),float(epoch_accuracy.cpu().detach().numpy()),float(test_loss.cpu().detach().numpy()), float(test_acc.cpu().detach().numpy())])

In [ ]:
i = 0
for times, (data) in enumerate(val_loader):
    print(times)
    i+=1
    if i == 5:
        break

# Submission

In [ ]:
# test_df['preds'] = preds.astype(int)
# submission = sample_submission.merge(test_df.rename(columns={'id': 'Id'})[['Id', 'preds']], on='Id').drop(columns='Predicted')
# submission['Predicted'] = submission['preds'].map(class_map)
# submission = submission.drop(columns='preds')
# submission.to_csv('submission.csv', index=False)
# submission.head()